# Automação Web e Busca de Informações com Python

#### Projetos: 

Precisamos calcular o valor a ser cobrado por produtos cotados em:
- Dólar
- Euro
- Ouro

Para isso, é necessário pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto deve ser cobrado pelos produtos, considerando uma margem de lucro escolhida.

Para que a automação web funcione, deve ser instalados:

- Biblioteca Selenium
- Chrome Webdriver

Ao final do processo, os valores atualizados serão exportados para uma planilha excel

### Ferramentas utilizadas

![Python](https://img.shields.io/badge/-Python-333333?style=for-the-badge&logo=Python)
![Pandas](https://img.shields.io/badge/-Pandas-333333?style=for-the-badge&logo=pandas)
![Selenium](https://img.shields.io/badge/-Selenium-333333?style=for-the-badge&logo=Selenium)
![Jupyter](https://img.shields.io/badge/-Jupyter-333333?style=for-the-badge&logo=jupyter)
![Excel](https://img.shields.io/badge/-Excel-333333?style=for-the-badge&logo=microsoftexcel)


In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True # also works
# nav = webdriver.Chrome(options=chrome_options)

nav = webdriver.Chrome()

# pesquisar cotação dolar
nav.get("https://www.google.com/")
nav.find_element("xpath",'//*[@id="APjFqb"]').send_keys("cotação dólar")
nav.find_element("xpath",'//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_dolar = nav.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# pesquisar cotação euro
nav.get("https://www.google.com/")
nav.find_element("xpath",'//*[@id="APjFqb"]').send_keys("cotação euro")
nav.find_element("xpath",'//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pegar cotacao ouro
nav.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = nav.find_element("id",'comercial').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

nav.quit()

5.0069
5.320031526
320.31


### Agora vamos atualizar a base de preços com as novas cotações

- Importando a base de dados

In [14]:
import pandas as pd

produtos_df = pd.read_excel("Produtos.xlsx")
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.403200,5403.145968,1.40,7564.40
1,Carro Renault,4500.00,Euro,6.521095,29344.927788,2.00,58689.86
2,Notebook Dell,899.99,Dólar,5.403200,4862.825968,1.70,8266.80
3,IPhone,799.00,Dólar,5.403200,4317.156800,1.70,7339.17
4,Carro Fiat,3000.00,Euro,6.521095,19563.285192,1.90,37170.24
5,Celular Xiaomi,480.48,Dólar,5.403200,2596.129536,2.00,5192.26
6,Joia 20g,20.00,Ouro,307.460000,6149.200000,1.15,7071.58


- Atualizando os preços e o cálculo do Preço Final

In [17]:
produtos_df.loc[produtos_df['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)

produtos_df['Preço Base Reais'] = produtos_df['Cotação'] * produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Ajuste'] * produtos_df['Preço Base Reais']
produtos_df["Preço Final"] = produtos_df["Preço Final"].map("{:.2f}".format)
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.006900,5006.849931,1.40,7009.59
1,Carro Renault,4500.00,Euro,5.320032,23940.141867,2.00,47880.28
2,Notebook Dell,899.99,Dólar,5.006900,4506.159931,1.70,7660.47
3,IPhone,799.00,Dólar,5.006900,4000.513100,1.70,6800.87
4,Carro Fiat,3000.00,Euro,5.320032,15960.094578,1.90,30324.18
5,Celular Xiaomi,480.48,Dólar,5.006900,2405.715312,2.00,4811.43
6,Joia 20g,20.00,Ouro,320.310000,6406.200000,1.15,7367.13


### Agora vamos exportar a nova base de preços atualizada

In [18]:
produtos_df.to_excel("Produtos Atualizado.xlsx", index=False)